## 셀레니움으로 스크래핑

In [63]:
import pandas as pd 
from selenium import webdriver
from bs4 import BeautifulSoup as bs 
import requests
from fake_useragent import UserAgent
pd.set_option('display.max_colwidth', None)

In [64]:
url = 'https://www.melon.com/chart/hot100/index.htm'
chrome_path = 'C:/Users/rnlay/.wdm/drivers/chromedriver/win64/118.0.5993.89/chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument(f'--executable-path={chrome_path}')

driver = webdriver.Chrome(options=options)

driver.implicitly_wait(10)
driver.get(url)

In [65]:
html = driver.page_source
soup = bs(html, 'html.parser')
song_soup = soup.select('tbody > tr')

In [66]:
song_lst = []

for song in song_soup:
    song_title = song.find('div', class_= 'ellipsis rank01').get_text()
    song_no = song.find('div', class_='ellipsis rank01').find('a')['href'].split("'")[2]
    song_artist = song.find('span', class_= 'checkEllipsis').get_text()
    song_album = song.find('div', class_= 'ellipsis rank03').get_text()
    song_likes = song.find('button', class_ = 'button_etc like').get_text()

    song_lst.append([song_no,
                     song_title.replace('\n', ''),
                     song_artist.replace('\n', ''),
                     song_album.replace('\n', ''),
                     song_likes.replace('\n', '').replace('좋아요총건수', '')])

In [67]:
driver.quit()

df = pd.DataFrame(song_lst, columns = ['곡ID', '곡명', '가수', '앨범', '좋아요'])

df['곡ID'] = df['곡ID'].str.extract(r'(\d+)')
df['순위'] = range(1, len(df) + 1)
df = df[['순위', '곡ID','곡명', '가수', '앨범', '좋아요']]
df.head()

,순위,곡ID,곡명,가수,앨범,좋아요
0,1,37138469,나는 아픈 건 딱 질색이니까,(여자)아이들,2,"88,969"
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,"98,596"
2,3,37194943,밤양갱,비비 (BIBI),밤양갱,"122,026"
3,4,37225604,EASY,LE SSERAFIM (르세라핌),EASY,"60,727"
4,5,37145732,Love wins all,아이유,The Winning,"150,340"


## 가사 추출

In [68]:
def get_lyrics(song_id):
    headers = {"user-agent": UserAgent().chrome}
    url = f"https://www.melon.com/song/detail.htm?songId={song_id}"
    res = requests.get(url, headers=headers)
    
    if res.status_code == 200:
        soup = bs(res.text, 'html.parser')
        lyrics_tag = soup.find('div', class_='lyric')
        
        if lyrics_tag:
            lyrics = lyrics_tag.get_text(separator=' ', strip=True)
            return lyrics

    return None

df2 = pd.DataFrame(df)
df2['가사'] = df2['곡ID'].apply(get_lyrics)
df2.head(2)

,순위,곡ID,곡명,가수,앨범,좋아요,가사
0,1,37138469,나는 아픈 건 딱 질색이니까,(여자)아이들,2,"88,969",None
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,"98,596",None


In [69]:
df2.to_csv("가사.csv", encoding="utf-8-sig")